<a href="https://colab.research.google.com/github/fmhirwa/ALU-skin-resilience-capstone/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Load

In [4]:
pip install pandas numpy geopandas matplotlib seaborn scikit-learn

.grid format to CSV

In [14]:
!apt-get update -qq
!apt-get install -y libeccodes-dev eccodes
!pip install cfgrib xarray

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package eccodes


In [19]:
# Temp Missing Engine
import xarray as xr
print(xr.backends.list_engines())

{'h5netcdf': <H5netcdfBackendEntrypoint>
  Open netCDF (.nc, .nc4 and .cdf) and most HDF5 files using h5netcdf in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.H5netcdfBackendEntrypoint.html, 'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


In [16]:
import xarray as xr
import pandas as pd

# Now the “cfgrib” engine should be available:
ds = xr.open_dataset('base_data.grib', engine='cfgrib')

# Flatten to a DataFrame:
df_env = ds.to_dataframe().reset_index()

# Example: rename and convert temperature if it’s called “t2m”:
df_env = df_env.rename(columns={'t2m': 'temperature_k'})
df_env['temperature_c'] = df_env['temperature_k'] - 273.15

# Keep only the columns you need:
df_env = df_env[['time', 'latitude', 'longitude', 'temperature_c']]

ValueError: unrecognized engine 'cfgrib' must be one of your download engines: ['h5netcdf', 'scipy', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [ ]:
# GRIB has time, latitude, longitude, etc., this will flatten it:
df_env = ds.to_dataframe().reset_index()

# Temperature is named 't2m' in the GRIB:
df_env = df_env.rename(columns={'t2m': 'temperature_k'})


In [ ]:
# Convert Kelvin to Celsius, if needed:
df_env['temperature_c'] = df_env['temperature_k'] - 273.15

# Keep only relevant columns (e.g., time, latitude, longitude, temperature_c):
df_env = df_env[['time', 'latitude', 'longitude', 'temperature_c']]


In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd  # may not need; just geospatial; gree credits nope
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
df_temp = pd.read_csv('era5_temperature_nairobi.csv') #1.2921° S, 36.8219° E
#f_air = pd.read_csv('air_quality_nairobi.csv')
#df_survey = pd.read_csv('survey_responses.csv')

In [ ]:
gdf_pollutants = gpd.read_file('sentinel5p_pm25.shp')


Cleaning And Preprocessing

In [ ]:
df.isnull().sum()
df.info()

In [ ]:
df_temp.dropna(subset=['temperature'], inplace=True)

In [ ]:
df_air['pm25'] = df_air['pm25'].fillna(df_air['pm25'].median())


In [ ]:
df_temp['date_time'] = pd.to_datetime(df_temp['date_time'])


In [ ]:
df_env = pd.merge(df_temp, df_air, on=['date_time', 'location_id'], how='inner')


Base Analysis

3.1 Stats

In [ ]:
df_env.describe()
df_survey.describe()


3.2 Time series Plot

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(df_env['date_time'], df_env['temperature_c'])
plt.title('Hourly Temperature')
plt.show()


3.3 Distribution

In [ ]:
sns.histplot(df_env['pm25_ug_m3'], bins=30)
plt.title('PM2.5 Distribution')
plt.show()


3.4 Correlation Matrix

In [ ]:
corr = df_env[['temperature_c','pm25_ug_m3','humidity','wind_speed']].corr()
sns.heatmap(corr, annot=True, fmt=".2f")
plt.title('Env Variable Correlations')
plt.show()


4. Features and Averages